# Visualization in python
There are several [visualization](https://en.wikipedia.org/wiki/Edge_detection) tools for 3D imagery have been developed with/for Python, for example
Matplotlib (Hunter, 2007), Mayavi (Ramachandran & Varoquaux, 2011), the [ipyvolume](https://github.com/maartenbreddels/ipyvolume/), the yt Project
(Turk et al., 2010), [ITK](https://itk.org/) (Johnson, McCormick, Ibanez 2015), and more recently [napari](http://napari.org). 

The 3 main challenges of available tools are: 
- working with large volumes: image sizes
- complexity of APIs: how many lines for simple rendering
- compatibility to ipynb as much of the prototyping is within Jupyter notebooks

This lesson will illustrate tools that work with large datasets by exploring 3D projections as well as data reduction techniques enabled through `scikit-image` functions. 

In [ ]:
import numpy as np
import napari
from skimage import exposure, io, measure

%gui qt

Let's read the original image from previous lesson and view it in napari:

In [ ]:
input_filename="../data/cells.tif"
orig = io.imread(input_filename)
print(orig.shape)

In [ ]:
viewer = napari.view_image(orig, name='orignal')

Let's now do some histogram equalization and view the results as a new layer in napari

In [ ]:
equalized = exposure.equalize_hist(orig)
print(equalized.shape)

In [ ]:
viewer.add_image(equalized, name='equalized')

Let's load and add the segmentation labels as a labels layer in napari

In [ ]:
from skimage.external import tifffile
inputfile = '../data/cells_interior_labels.tiff'
relabeled = tifffile.imread(inputfile)
print(relabeled.shape)

In [ ]:
viewer.add_labels(relabeled, name='labeled')

Let's inspect the data that we've added to napari

In [ ]:
viewer.layers

In [ ]:
viewer.layers['equalized'].data

In [ ]:
viewer.layers['equalized'].data = viewer.layers['equalized'].data / 2

## Surface of 3D objects
`skimage.measure.regionprops` **==>** `skimage.measure.marching_cubes` **==>** `skimage.measure.mesh_surface_area`

Perimeter measurements are not computed for 3D objects. The 3D extension of perimeter is surface area. We can measure the surface of an object by generating a surface mesh with 
- `skimage.measure.marching_cubes` returns 2 arrays: spatial coordinates for V unique mesh vertices and F faces that define triangular faces via referencing vertex indices from V. This algorithm specifically outputs triangles, so each face has exactly three indices.
- Marching cubes: extract a polygonal mesh of an isosurface from a three-dimensional discrete scalar field (voxels); basic steps are: divide the input volume into discrete set of cubes; each cube contains a piece of a given isosurface; a triangular mesh approximates the behavior of the trilinear interpolant in the interior cube.

In [ ]:
selected_cell = 3

regionprops = measure.regionprops(relabeled.astype('int'))
# skimage.measure.marching_cubes expects ordering (row, col, pln)
volume = (relabeled == regionprops[selected_cell].label).transpose(1, 2, 0)

verts, faces, _, values = measure.marching_cubes_lewiner(volume, level=0, spacing=(1.0, 1.0, 1.0))

Now we will compute the surface area of the mesh with `skimage.measure.mesh_surface_area`.

In [ ]:
surface_area_pixels = measure.mesh_surface_area(verts, faces)
print("Surface area (total pixels): {:0.2f}".format(surface_area_pixels))

In [ ]:
viewer.add_surface((verts[:, [2, 0, 1]], faces, np.random.random(len(verts))), name='surface')

In [ ]:
viewer.layers

## Looking at max projections

Large datasets are a gift and a curse: while high-resolution imaging brings details about microstructures, the file sizes can quickly became an obstacle to analysis, particularly at your laptop. Here are some tips on how to proceed with your first explorations of your imagesets

In [ ]:
max_projection = orig.max(axis=0)
max_projection_labeled = relabeled.max(axis=0)

In [ ]:
viewer = napari.view_image(max_projection, name='max projection', title='2d viewer')
viewer.add_labels(max_projection_labeled, name='max projection labels')

## Visualizing filer results

In [ ]:
from skimage import filters  # skimage's filtering module

In [ ]:
plane_sobel_h = filters.sobel_h(max_projection)  # Horizontal Sobel.
plane_sobel_v = filters.sobel_v(max_projection)  # Vertical Sobel.
plane_roberts = filters.roberts(max_projection)  # Roberts.
plane_prewitt = filters.prewitt(max_projection)  # Prewitt.
plane_scharr = filters.scharr(max_projection)  # Scharr.

In [ ]:
viewer.add_image(plane_sobel_h, name='Horizontal Sobel')
viewer.add_image(plane_sobel_v, name='Vertical Sobel')
viewer.add_image(plane_roberts, name='Roberts')
viewer.add_image(plane_prewitt, name='Prewitt')
viewer.add_image(plane_scharr, name='Scharr')

## Interactive segmentation

In [ ]:
from skimage import morphology  # skimage's morphological submodules.
from skimage import feature  # skimage's feature submodule.
from scipy import ndimage

Let's repeat the binarization process and distance calculation that we did at the end of the last lesson

In [ ]:
plane_binary = max_projection >= filters.threshold_li(max_projection)
plane_remove_holes = morphology.remove_small_holes(plane_binary, 60)
plane_remove_objects = morphology.remove_small_objects(plane_remove_holes, min_size=50)
plane_distance = ndimage.distance_transform_edt(plane_remove_objects)

In [ ]:
viewer = napari.view_image(max_projection, name='original', title='segmentation')
viewer.add_image(plane_binary, name='binary')
viewer.add_image(plane_remove_holes, name='remove holes')
viewer.add_image(plane_remove_objects, name='remove objects')
viewer.add_image(plane_distance, name='distances')

Let's now finish the segmentation as before

In [ ]:
smooth_distance = filters.gaussian(plane_distance, sigma=10)
peak_local_max = feature.peak_local_max(
    smooth_distance,
    footprint=np.ones((7, 7), dtype=np.bool),
    indices=False,
    labels=measure.label(plane_remove_objects)
)
peaks = np.nonzero(peak_local_max)

plane_markers = measure.label(peak_local_max)

plane_labels = morphology.watershed(
    -smooth_distance, 
    plane_markers, 
    mask=plane_remove_objects
)

In [ ]:
viewer.add_image(smooth_distance, name='smooth distances')
viewer.add_points(np.array(peaks).T, name='peaks', size=5)
viewer.add_labels(plane_labels, name='labels')

Now lets update the labeling based on our new points

In [ ]:
def update_labeling(viewer):
    viewer.status = 'updating labels'
    peak_data = np.round(viewer.layers['peaks'].data).astype(int)
    new_peaks = (peak_data.T[0], peak_data.T[1])
    peak_local_max = np.zeros(max_projection.shape, dtype=bool)
    peak_local_max[new_peaks] = 1
    plane_markers = measure.label(peak_local_max)
    new_plane_labels = morphology.watershed(
        -smooth_distance, 
        plane_markers, 
        mask=plane_remove_objects
    )
    viewer.layers['labels'].data = new_plane_labels

In [ ]:
update_labeling(viewer)

Now lets bind this function to a keypress in the viewer

In [ ]:
viewer.bind_key('Control-U', update_labeling)

# Going beyond

[1] Segmentation of 3-D tomography images with Python: http://emmanuelle.github.io/segmentation-of-3-d-tomography-images-with-python-and-scikit-image.html

[2] Image processing with Dask arrays: https://dask-image.readthedocs.io/en/latest/